In [1]:
import pandas_profiling
%matplotlib inline
import pandas as pd
import datetime as dt
import re
import matplotlib.pyplot as plt

In [2]:
# Import dataframe containing the raw data
pp_data = pd.read_csv("pp_data_1.csv")

In [3]:
print("The raw data includes the following columns taken directly from r/progress_pics")
print(pp_data.columns)

The raw data includes the following columns taken directly from r/progress_pics
Index(['title', 'score', 'id', 'url', 'num_comments', 'created_utc', 'author',
       'permalink', 'timestamp'],
      dtype='object')


### Titles in the progress_pics subreddit are supposed to be formatted in the following manner:  
Gender/Age/Height \[Weight Before > Weight After = Total amount lost\](Time period in months) Personal Title

### Sample entry  
"M/26/6'0 \[190lbs&gt;175lbs=15lbs loss\] -2.5 years. All natural hard work."

### Issues with processing  
The formatting rules are not followed  
Some posters use metric units or include English and metric units


## Processing the title 

In [4]:
# function to process the title column and extract the sex, age, height, and starting and ending weights.  
# In the future, extend the function to also extract the "time period in months" and the "Personal Title" 

def process_stats_regex_ver1(s):
    stats = {}
    clean_s = s.upper().lstrip().replace(' ', '')
    #print(clean_s)
    regex = re.compile(r"^([^/]+)(?:/)([^/]+)(?:/)([^/]+)(?:[(\[)(\()(/)])(\d+)(?:.*)(?:[&GTL;]{4}|[TO]{2}|-)(\d+)")
    result = regex.search(clean_s)
    if result:
        #print((result.groups()))
        return result.group(1), result.group(2), result.group(3), result.group(4), result.group(5)
    else:
        #return "unknown"
        return "unknown", "unknown", "unknown", "unknown", "unknown"       

In [5]:
# add new columns: 'sex', 'age', 'height', 'start_weight', 'end_weight'
pp_data = pp_data.reindex(columns=['title', 'sex', 'age', 'height', 'start_weight', 'end_weight', 'score', 'timestamp','id', 'num_comments', 'created_utc', 'author', 'permalink'])

In [6]:
# populate the new columns using the process_stats_regex_ver1 function
pp_data[["sex", "age", "height", "start_weight", "end_weight"]] = pp_data.apply(lambda row: process_stats_regex_ver1(row["title"]), axis=1, result_type="expand")

In [7]:
# Many of the entries that were processed incorrectly can be identified by having
# a height entry that contains portions of the title past the height regions.  Started with 27464 entries, 20162 have a
# height <= 6 characters so were processed correctly.  Wrote another processing funtion that fixes a small number of 
# the incorrectly processed entries.  

def process_stats_regex_ver2(s):
    stats = {}
    clean_s = s.upper().lstrip().replace(' ', '')
    #print(clean_s)
    regex = re.compile(r"^([^/]+)(?:/)([^/]+)(?:/)([^/]+)(?:[(\[)(\()(/)])(\d+)(?:[&GTL;]{4}|[TO]{2}|-)(\d+)")
    result = regex.search(clean_s)
    if result:
        #print((result.groups()))
        return result.group(1), result.group(2), result.group(3), result.group(4), result.group(5)
    else:
        #return "unknown"
        return "unknown", "unknown", "unknown", "unknown", "unknown"

In [8]:
# spilt df into those with correct and incorrect height entries
just_right = pp_data[pp_data["height"].str.len() <=6]
too_long = pp_data[pp_data["height"].str.len() > 6]

In [9]:
# reprocess the too long entries with process_stats_regex_ver2 and pull out those with the correct height entry

too_long[["sex", "age", "height", "start_weight", "end_weight"]] = too_long.apply(lambda row: process_stats_regex_ver2(row["title"]), axis=1, result_type="expand")
now_just_right = too_long[too_long["height"].str.len() <= 6]

# How to fix this error?

/Users/ajwright/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [10]:
# concat the two df containing correctly processed titles and reindex 

frames = [just_right, now_just_right]
pp_data = pd.concat(frames)
pp_data.reset_index(inplace=True, drop=True)

In [11]:
# This misprocessing is actually a large problem that also results in the wrong number being entered in the "end_weight"
# column.  Need to address! 

## Cleaning the sex column

In [12]:
#function that processes the sex column and returns M, F or unknown if the sex column contains and age (2 digits) 
# and cannot be processed
def clean_sex(s):
    if s == "M" or s == "F":
        return s
    elif s == "MALE":       
        return "M"
    elif s == "FEMALE":
        return "F"
    elif s.isdigit():
        return "unknown"
    else:
        return s[-1]

In [13]:
# apply clean_sex function to the df; store results in "clean_sex" column then reorder the columns of the df

pp_data["clean_sex"] = pp_data["sex"].apply(clean_sex)
pp_data = pp_data.reindex(columns=['title', 'sex', 'clean_sex','age', 'height', 'start_weight', 'end_weight', 'score', 'timestamp','id', 'num_comments', 'created_utc', 'author', 'permalink'])


In [14]:
# seperate df on the basis of if clean_sex is "unknown" or not - 233 are unknown meaning they have a digit

sex_known = pp_data[pp_data["clean_sex"] != 'unknown']
sex_unknown = pp_data[pp_data["clean_sex"] == 'unknown']

In [15]:
# Due to poster error, when the clean_sex column contains an age, then the age column usually contains the sex.  
# Switch the sex and age column labels in the sex_unknown df

sex_unknown.columns = ['title', 'age', 'clean_sex', 'sex', 'height', 'start_weight',
       'end_weight', 'score', 'timestamp', 'id', 'num_comments', 'created_utc',
       'author', 'permalink']

In [16]:
# Process the "new" sex column using clean_sex
sex_unknown.loc[:, "clean_sex"] = sex_unknown.loc[:, "sex"].apply(clean_sex)

# How to get rid of this error?

/Users/ajwright/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
# Identify rows in unknown_sex where the "clean_sex" now contains an "M" or "F

cleaned_unknown_sex = sex_unknown[(sex_unknown["clean_sex"] == "M")|(sex_unknown["clean_sex"] == "F")]

In [18]:
# Switch the column order to match the sex_known df before concatenation 

cleaned_unknown_sex = cleaned_unknown_sex.reindex(columns=['title', 'sex', 'clean_sex', 'age', 'height', 'start_weight',
       'end_weight', 'score', 'timestamp', 'id', 'num_comments', 'created_utc',
       'author', 'permalink'])

In [19]:
# Concatenate the df and reindex

frames = [sex_known, cleaned_unknown_sex]
pp_data = pd.concat(frames)
pp_data.reset_index(inplace=True, drop=True)

In [20]:
# Seperate all the entries that have only M and F in the clean_sex column

pp_data = pp_data[(pp_data["clean_sex"] == "M")|(pp_data["clean_sex"] == "F")]

## Cleaning the age column

In [21]:
# Turns out 109 entries have ages in the heights columns.  Seperate out those entries into the age_info df, switch 
# the columns, update the original df with the new info. 

pattern = r"([0-9]{2})"
age_info = pp_data[(pp_data["height"].str.match(pattern))&(pp_data["height"].str.len() == 2)]
age_info.columns = ['title', 'sex', 'clean_sex', 'height', 'age', 'start_weight',
       'end_weight', 'score', 'timestamp', 'id', 'num_comments', 'created_utc',
       'author', 'permalink']
pp_data.update(age_info)

In [22]:
# only 79 entries with age length greater than 2 so filter those out, also filter out any that do not have 2 digits. 

pp_data = pp_data[pp_data["age"].str.len() == 2]
pattern = r"(^[0-9]{2}?)"
pp_data = pp_data[pp_data["age"].str.match(pattern)]



In [23]:
# convert age from strings to integers

pp_data["age"] = pd.to_numeric(pp_data["age"], errors='coerce')

## Cleaning the height column

In [24]:
# Strip out all non digit characters leaving behind just the numbers

def number_height(s):
    chars = list(s)
    digit_chars = [c for c in chars if c.isdigit()]
    number_s = "".join(digit_chars)
    return number_s

In [25]:
# apply number_height function to each height entry and store the results in num_height column

pp_data["num_height"] = pp_data["height"].apply(number_height)

In [26]:
# Measurements in feet will start with 3, 4, 5, while measurements in cm will start with 1.  The function below
# converts num_height to the height in inches. 

def height_inches(s):
    ft_list = ["4", "5", "6", "7"]
    if s == '':
        return "unknown"
    elif s[0] == "1":
        return int(s) * 0.39370079 
    elif s[0] in ft_list:
        if len(s) == 1:
            return int(s) *12
        elif len(s) == 3 and s[2] == "5":
            return  (int(s[0]) * 12) + int(s[1]) + 0.5
        else:
            return (int(s[0]) * 12) + int(s[1:])
    else:
        return "unknown"


In [27]:
# apply height_inches function to each num_height entry and store the results in height_in column

pp_data["height_in"] = pp_data["num_height"].apply(height_inches)

In [28]:
# convert height_in from strings to integers

pp_data.loc[:,"height_in"] = pd.to_numeric(pp_data["height_in"], errors='coerce')

In [29]:
# remove entries with height outliers (< 4'6'' and > 7'1')

pp_data = pp_data[(pp_data["height_in"] > 54) & (pp_data["height_in"] < 85)]

In [30]:
# reorder the columns 

pp_data = pp_data.reindex(columns=['title', 'sex', 'clean_sex', 'age', 'height', 'num_height', 'height_in','start_weight',
       'end_weight', 'score', 'timestamp', 'id', 'num_comments', 'created_utc',
       'author', 'permalink'])

## Cleaning the weight columns 

In [31]:
# Create 2 columns to hold the processed weight columns "new_start_weight" and "new_end_weight"

pp_data = pp_data.reindex(columns=['title', 'sex', 'clean_sex', 'age', 'height', 'num_height', 'height_in',
       'start_weight', 'new_start_weight','end_weight', 'new_end_weight', 'score', 'timestamp', 'id',
       'num_comments', 'created_utc', 'author', 'permalink'], fill_value=0)

In [32]:
# Change dtypes of the start_weight and end_weight columns

pp_data.loc[:,"start_weight"] = pd.to_numeric(pp_data["start_weight"], errors='coerce')
pp_data.loc[:,"end_weight"] = pd.to_numeric(pp_data["end_weight"], errors='coerce')

In [33]:
# to identify weights in kgs, looks for kg in the title using the regex pattern.  If found, process the start_weight
# and end_weight like they are kg.  Multiple weights by 2.20462 to convert to lbs and store in the new weight columns

pattern = r"=\s*\d+\s*KG"
pp_data.loc[pp_data.title.str.contains(pattern, case=False), "new_start_weight"] = pp_data.loc[pp_data.title.str.contains(pattern, case=False), "start_weight"] * 2.20462
pp_data.loc[pp_data.title.str.contains(pattern, case=False), "new_end_weight"] = pp_data.loc[pp_data.title.str.contains(pattern, case=False), "end_weight"] * 2.20462


In [34]:
# the regex pattern missed some weights in kgs so also screen for start_weight < 130 and the letters "kg" anywhere 
# in the title (along the new weight column == 0 indicating it wasn't processed the first time).  Multiple by 2.20462 
# to convert to lbs and store in the new weight columns.

pp_data.loc[(pp_data.start_weight < 130) & pp_data.title.str.contains("kg", case=False) & (pp_data.new_start_weight == 0), "new_start_weight"] = pp_data.loc[(pp_data.start_weight < 130) & pp_data.title.str.contains("kg", case=False) & (pp_data.new_start_weight == 0), "start_weight"] * 2.20462
pp_data.loc[(pp_data.start_weight < 130) & pp_data.title.str.contains("kg", case=False) & (pp_data.new_end_weight == 0), "new_end_weight"] = pp_data.loc[(pp_data.start_weight < 130) & pp_data.title.str.contains("kg", case=False) & (pp_data.new_end_weight == 0), "end_weight"] * 2.20462


In [35]:
# For all entries not processed above, copy the starting weights to the new weight columns with no modifications. 

pp_data.loc[(pp_data.new_start_weight == 0), "new_start_weight"] = pp_data.loc[(pp_data.new_start_weight == 0), "start_weight"]
pp_data.loc[(pp_data.new_end_weight == 0), "new_end_weight"] = pp_data.loc[(pp_data.new_end_weight == 0), "end_weight"]

In [36]:
# trim off new_start_weight entries over 1000 lbs due to processing errors 

pp_data = pp_data[pp_data["new_start_weight"] <= 1000]

In [37]:
# trim off new_end_weight entries over 1000 lbs and less than 70 lbs due to processing errors

pp_data = pp_data[pp_data['new_end_weight'] >=70]
pp_data = pp_data[pp_data['new_end_weight'] < 1000]

## Calculating weight gained/lost ("weight_diff" column)

In [38]:
# populate 'weight_diff' column by subtracting new_end_weight from new_start_weight

pp_data["weight_diff"] = pp_data["new_start_weight"] - pp_data["new_end_weight"]

In [39]:
# change column order

pp_data = pp_data.reindex(columns=['title', 'sex', 'clean_sex', 'age', 'height', 'num_height', 'height_in',
       'start_weight', 'new_start_weight', 'end_weight', 'new_end_weight', 'weight_diff',
       'score', 'timestamp', 'id', 'num_comments', 'created_utc', 'author',
       'permalink'])

## Adding date time columns 

In [40]:
# Change timestamp from object to datetime

pp_data["timestamp"] = pd.to_datetime(pp_data["timestamp"]) 

In [41]:
# Create columns to hold different date and time metrics 

pp_data["month"] = pp_data["timestamp"].dt.month
pp_data["year"] = pp_data["timestamp"].dt.year
pp_data["day"] = pp_data["timestamp"].dt.day
pp_data["dayofweek"] = pp_data["timestamp"].dt.dayofweek
pp_data["date"] = pp_data["timestamp"].dt.date
pp_data["time"] = pp_data["timestamp"].dt.time

In [42]:
# change column order

pp_data = pp_data.reindex(columns=['title', 'clean_sex', 'age',  'height_in',
       'new_start_weight', 'new_end_weight',
       'weight_diff', 'score', 'num_comments', 'id', 'author', 'permalink', 'timestamp',
       'date', 'time', 'year', 'month', 'day',  'dayofweek',
       'sex', 'height', 'num_height', 'start_weight', 'end_weight', 'created_utc'])

In [43]:
# rename some columns so the final versions have simpler names 

pp_data.columns = (['title', 'sex', 'age', 'height_in', 'start_weight',
       'end_weight', 'weight_diff', 'score', 'num_comments', 'id',
       'author', 'permalink', 'timestamp', 'date', 'time', 'year', 'month',
       'day', 'dayofweek', 'ori_sex', 'height', 'num_height', 'ori_start_weight',
       'ori_end_weight', 'created_utc'])

## Save processed dataframe to a cvs file

In [44]:
pp_data.to_csv("pp_data_2018_comp_final.csv", index=False)

In [45]:
pp_data.head()

,title,sex,age,height_in,start_weight,end_weight,weight_diff,score,num_comments,id,...,year,month,day,dayofweek,ori_sex,height,num_height,ori_start_weight,ori_end_weight,created_utc
0,"M/22/6'3"" [290lbs &gt; 185lbs = 105lbs] (2 yea...",M,22,75.0,290.0,185.0,105.0,246.0,10.0,7ru6qv,...,2018,1,20,5,M,"6'3""",63,290,185,1.516493e+09
2,F/23/5’0” [260 &gt; 218 = 42lbs] (12 months) F...,F,23,60.0,260.0,218.0,42.0,210.0,5.0,7rvemg,...,2018,1,20,5,F,5’0”,50,260,218,1.516506e+09
3,F/24/5’3” (160 SW &gt; 135 CW &gt; 125 GW) 25l...,F,24,63.0,160.0,125.0,35.0,1.0,1.0,7rvnzq,...,2018,1,20,5,F,5’3”,53,160,125,1.516509e+09
4,F/48/5’6” [326 lbs &gt; 180lbs= 146 lbs lost] ...,F,48,66.0,326.0,180.0,146.0,3163.0,144.0,7rvoaj,...,2018,1,20,5,F,5’6”,56,326,180,1.516509e+09
5,"F/38/5'7"" [204lbs &gt; 141lbs = 63lbs] (1 year...",F,38,67.0,204.0,141.0,63.0,212.0,24.0,7rvpuq,...,2018,1,20,5,F,"5'7""",57,204,141,1.516510e+09
